# Managing Conversational Memory with Flotorch SDK

This notebook provides a comprehensive guide to using the `FlotorchMemory` and `FlotorchAsyncMemory` classes. These clients allow you to perform the full lifecycle of memory operations (Create, Read, Update, and Delete) for storing and managing conversational context.

### Prerequesit
Configure memory provider, API key in Flotroch console (https://console.flotorch.cloud/)

### Key Objectives:
- Initialize both the synchronous and asynchronous memory clients.
- Perform **Add, Search, Update,** and **Delete** operations using the synchronous `FlotorchMemory` client.
- Perform the same operations using the asynchronous `FlotorchAsyncMemory` client.

## 1. Setup and Initialization

First, we'll set up the environment by loading the necessary API credentials. We will then initialize two separate clients:

- **`FlotorchMemory`**: For standard, blocking operations.
- **`FlotorchAsyncMemory`**: For non-blocking operations in concurrent applications.

In [ ]:
%pip install --pre flotorch[sdk]

In [ ]:
FLOTORCH_API_KEY = "<flotorch api key>"
FLOTORCH_BASE_URL = "<flotroch gateway base url>" # eg: https://gateway.flotorch.cloud"
MEMORY_PROVIDER = "<flotorch memory provider>"

In [ ]:
# import necessary libraries
import uuid
from flotorch.sdk.memory import FlotorchMemory, FlotorchAsyncMemory

print("Imported necessary libraries successfully")

In [ ]:
# Initialize synchronous memory client
sync_memory = FlotorchMemory(
    api_key = FLOTORCH_API_KEY,
    base_url= FLOTORCH_BASE_URL,
    provider_name = MEMORY_PROVIDER
)

# Test parameters for sync operations
sync_session_id = str(uuid.uuid4())
sync_user_id = "sync_user_001"
sync_agent_id = "sync_agent_001"
sync_source = "demo_application_sync"

print("sync_memory is initialized")

In [ ]:
# Initialize asynchronous memory client
async_memory = FlotorchAsyncMemory(
    api_key = FLOTORCH_API_KEY,
    base_url = FLOTORCH_BASE_URL,
    provider_name = MEMORY_PROVIDER
)

# Test parameters for async operations
async_session_id = str(uuid.uuid4())
async_user_id = "async_user_001"
async_agent_id = "async_agent_001"
async_source = "demo_application_async"

print("async_memory is initialized")

## 2. Synchronous Memory Operations

The `FlotorchMemory` class provides simple, blocking methods for managing memory. Use this for standard scripts or applications where operations can be performed sequentially.

### Add, Search, Update, and Delete (Sync)

In [ ]:
# === 1. Add Memory (Sync) ===
memory_content = "User mentioned they prefer Python over Java for data science projects"
add_response = sync_memory.add(
    messages=[{"role": "user", "content": memory_content}],
    userId=sync_user_id,
    agentId=sync_agent_id,
    sessionId=sync_session_id,
    metadata={"source": sync_source, "category": "preferences"}
)
sync_memory_id = add_response["data"][0]["id"]
print(f"Memory Added (ID: {sync_memory_id})\n")

In [ ]:
# === 2. Search Memory (Sync) ===
search_response = sync_memory.search(userId=sync_user_id, sessionId=sync_session_id)
print(f"Memory Searched: Found {len(search_response['data'])} entr(y/ies)")
print(f"Content: {search_response['data'][0]['content']}\n")

In [ ]:
# === 3. Update Memory (Sync) ===
updated_memory_content = "User strongly prefers Python for data science and machine learning."
update_response = sync_memory.update(
    sync_memory_id,
    content=updated_memory_content,
    metadata={"source": sync_source, "importance": 0.9}
)
print(f"Memory Updated")
print(f"New Content: {update_response['data']['content']}\n")


In [ ]:
# === 4. Delete Memory (Sync) ===
delete_response = sync_memory.delete(sync_memory_id)
print(f"Memory Deleted (ID: {sync_memory_id})")

## 3. Asynchronous Memory Operations

The `FlotorchAsyncMemory` class provides non-blocking methods that must be used with `await` inside an `async` function. This is ideal for high-performance applications that handle many operations concurrently.

### Add, Search, Update, and Delete (Async)

In [ ]:
# === 1. Add Memory (Async) ===
memory_content = "User is interested in learning TensorFlow for deep learning applications"
add_response = await async_memory.add(
    messages=[{"role": "user", "content": memory_content}],
    userId=async_user_id,
    agentId=async_agent_id,
    sessionId=async_session_id,
    metadata={"source": async_source, "category": "learning"}
)
async_memory_id = add_response['data'][0]['id']
print(f"Memory Added (ID: {async_memory_id})\n")


In [ ]:
# === 2. Search Memory (Async) ===
search_response = await async_memory.search(userId=async_user_id, sessionId=async_session_id)
print(f"Memory Searched: Found {len(search_response['data'])} entr(y/ies)")
print(f"Content: {search_response['data'][0]['content']}\n")

In [ ]:
# === 3. Update Memory (Async) ===
updated_memory_content = "User is actively learning TensorFlow and PyTorch."
update_response = await async_memory.update(
    async_memory_id,
    content=updated_memory_content,
    metadata={"source": async_source, "technology": "tensorflow,pytorch"}
)
print(f"✓ Memory Updated")
print(f"   New Content: {update_response['data']['content']}\n")


In [ ]:
# === 4. Delete Memory (Async) ===
await async_memory.delete(async_memory_id)
print(f"Memory Deleted (ID: {async_memory_id})")    


## Summary

This guide covered the core CRUD (Create, Read, Update, Delete) operations for managing conversational memory using both synchronous and asynchronous clients.

### API at a Glance:

| Operation | Synchronous (`FlotorchMemory`) | Asynchronous (`FlotorchAsyncMemory`) |
| :--- | :--- | :--- |
| **Create** | `.add(...)`                             | `await .add(...)`                             |
| **Read** | `.search(...)`                          | `await .search(...)`                          |
| **Update** | `.update(id, ...)`                      | `await .update(id, ...)`                      |
| **Delete** | `.delete(id)`                           | `await .delete(id)`                           |

### Key Points to Remember:

1.  **Choose the Right Client**: Use `FlotorchMemory` for simple, sequential scripts and `FlotorchAsyncMemory` for concurrent applications like web servers.
2.  **Mandatory Identifiers**: All operations rely on `userId`, `agentId`, and `sessionId` to correctly scope and organize memories.
3.  **Required Metadata**: The `source` field is **mandatory** within the `metadata` dictionary when adding or updating memories.
4.  **Immutability**: The `delete` operation is permanent and cannot be undone. Always use with caution.